Consertarei primeiro as chaves 'key' das tags

In [1]:
# -*- coding: utf-8 -*-
import sqlite3
import pprint
import re
import pprint

Rnormal = re.compile(r'^([\w\. _])+',re.U)
Rproblemchars = re.compile(r'[=\+/&<>;\'"\?%#$@\,\. \t\r\n]')
Ruppercase_incorrect=re.compile(r'[A-Z][A-Z]+',re.U)
Rspace = re.compile(r'^([\w\. _])*[ ]',re.U)

def update_problems_keys(name):
    '''Função para corrigir os nomes nas colunas keys'''
    if(Rproblemchars.search(name)!=None):
        if(name==u'Unid.Hab'):
            return u'unidade_habitacional'
        elif(Rspace.search(name)!=None):
            return Rspace.search(name).group()[:-1]+'_'+ name[(Rspace.search(name)).end():]
        else:
            print name+" Problem",Rspace.search(name)
    if(not name.islower()):
        if(name=='FID' or name=='IBGE' or name=='BR' or name=='CE'):
            return name
        elif(name==u'AProjetada'):
            return u'área_projetada'
        elif(name=='AConst'):
            return u'área_construída'
        elif(name=='GEOCODIGO'):
            return u'geocodigo'    
        elif(name=='UNIDADES_HABITACIONAIS'):
            return u'unidades_habitacionais'        
        else:
            return name.lower()
    return name
db=sqlite3.connect("GrandeFortaleza.db")
c=db.cursor()
QUERY="""SELECT key,value FROM ways_tags;"""
c.execute(QUERY)
rows=c.fetchall()
for key,valor in rows:
    alterado=update_problems_keys(key)
 #   print alterado
    if(key!=alterado):
        CHANGE="UPDATE ways_tags SET key = replace(key, %s, %s);" % ("'"+key+"'","'"+alterado+"'")
        print CHANGE
        c.execute(CHANGE)
QUERY="""SELECT key,value FROM nodes_tags;""" 
c.execute(QUERY)
rows=c.fetchall()
for key,valor in rows:
    alterado=update_problems_keys(key)    
    update_problems_keys(key)
    if(key!=alterado):
        CHANGE="UPDATE nodes_tags SET key = replace(key, %s, %s);" % ("'"+key+"'","'"+alterado+"'")
        print CHANGE
        c.execute(CHANGE)
db.commit()
db.close()

Agora que não temos mais valores estranhos nas chaves passarei para os endereços para fins educativos usarei outro método aqui. importarei como um dataframe do pandas. Alterarei o DataFrame e o reinserirei na database.


In [5]:
import pandas as pd
from collections import defaultdict
street_type_re = re.compile(r'^\b\S+\.?', re.IGNORECASE)

R2problemchars = re.compile(r'[=\+/&<>;\'"\?%#$@\,\t\r\n]')
expected = [u"Rua",u"Avenida",u"Beco",u"Alameda",u"Acesso",u"Travessa", u"CE",u"BR",u"4º",u'Campus',u'Praça','Vila']

def audit_street_type(street_types, street_name):
    """Audita para verificar se falta algum prefixo"""
    m = street_type_re.search(street_name)
    if m:
        street_type = m.group()
        if street_type not in expected:
            street_types[street_type].add(street_name)
            
            
mapping = { u"Av.": u"Avenida",
            u"Av": u"Avenida",
            u"Tv.": u"Travessa",
            u"R.": u"Rua",
            u"Ce": u"CE",
            u"Br": u"BR"
            }
def update_problems_addr(name):
    '''Função para corrigir os nomes nas colunas value do tipo addr'''
    name=name.title()
    if(name==u'Costa Barros'):
        return u'Avenida Costa Barros'
    elif(name==u'Barbosa de Freitas'):
        return u'Avenida Barbosa de Freitas'
    elif(name==u'Joaquim Nabuco'):
        return u'Rua Joaquim Nabuco'
    elif(name==u'Eng\xb0 Sandoval S\xe1'):
        return u'Rua Engenheiro Sandoval'
    elif(name==u'2'):
        return u'Rua 2'
    elif(name==u'Alu\xedsio Mamede'):
        return u'Rua Aluísio Mamede'
    elif(name==u'Titan'):
        return u'Rua Titan'
    elif(name==u'Washington Soares'):
        return u'Avenida Washington Soares'    
    elif(name==u'José Severino'):
        return u'Rua José Severino'
    elif(name==u'Rua Nove,'):
        return u'Rua Nove'
    elif(name==u'João Paula Costa'):
        return u'Rua João Paula Costa'
    elif(name==u'Maria Teixeira Joca'):
        return u'Rua Maria Teixeira Joca'
    elif(name==u'Santa Teresinha'):
        return u'Rua Santa Teresinha'
    elif(name==u'Santa Teresinha'):
        return u'Rua Santa Teresinha'
    elif(name==u'Alphaville Fortaleza'):
        return u'Avenida Litorânea'
    elif(street_type_re.search(name).group() not in expected):
        rua = street_type_re.search(name)
        print name
        name= re.sub(street_type_re,mapping[rua.group()],name)
    return name
def audit_street(name):
    db=sqlite3.connect("GrandeFortaleza.db")
    QUERY='SELECT * FROM '+name
    nodes_tags=pd.read_sql(QUERY,db)
    count=0
    street_types = defaultdict(set)
    for valor,tipo,chave in nodes_tags[['value', 'type','key']].values:
        if(tipo=='addr'and chave=='street'):   
            alterado=update_problems_addr(valor)
            if(valor!=alterado):
                nodes_tags['value'].values[count]=alterado
            audit_street_type(street_types, alterado) 
        count+=1
    nodes_tags.to_sql(name,db, if_exists='replace', index=False)
    #print (street_types)
    #print nodes_tags.head()
    db.commit()
    db.close()

In [6]:
audit_street(u'nodes_tags')

Farei o mesmo com o ways_tags agora.

In [7]:
audit_street(u'ways_tags')

Ce 090


O resultado Ce apesar de aparecer como se ainda estivesse errado somente está aí pois eu faço capitalize no inicio da análise. porém ele é corrigido logo depois de printado.

irei descartar agora as tags com 'key'=fixme

In [12]:
db=sqlite3.connect("GrandeFortaleza.db")
c=db.cursor()
CHANGE="DELETE FROM ways_tags WHERE key='fixme';"
c.execute(CHANGE)
CHANGE="DELETE FROM nodes_tags WHERE key='fixme';"
c.execute(CHANGE)
db.commit()
db.close()